In [1]:
embedding_type = "ada"

In [2]:
import pandas as pd
import json

with open(file=f"./{embedding_type}.json", mode="r") as f:
    data = json.load(f)

df = pd.json_normalize(data, max_level=1).T  
  
# Print the DataFrame  
df.head(2)

,0
1.move up,"{'closest_pattern': 'Move up', 'predicted_feat..."
1.closer to the plate,"{'closest_pattern': 'Stay close to plate', 'pr..."


In [3]:
with open(file=f"./{embedding_type}.json", mode="r") as f:
    data = json.load(f)

corrections = {}
for id in data:
    correction = data[str(id)].pop("correction") 
    corrections[id] = correction


In [4]:
df_corr = pd.DataFrame.from_dict(corrections, orient='columns').T
df_corr.reset_index(inplace=True)  
df_corr.head(2)

,index,predicted,truth
0,1,"[microwave distance increase, plate distance d...","[microwave distance decrease, plate distance d..."
1,2,"[cup distance decrease, z-cartesian increase]","[cup distance decrease, microwave distance dec..."


In [5]:
rows = []  
for id, values in data.items():  
    segment = 1  
    for cmd, cmd_values in values.items():
        if isinstance(cmd_values, dict):  
            row = {"id": id,"segment": segment, "cmd": cmd}  
            row.update(cmd_values)  
            rows.append(row)  
            segment += 1 
  
df = pd.DataFrame(rows) 
df.head(3)

,id,segment,cmd,closest_pattern,predicted_feature,confidence_score
0,1,1,move up,Move up,Z-cartesian increase,0.97794
1,1,2,closer to the plate,Stay close to plate,plate distance decrease,0.91750
2,1,3,microwave,Avoid microwave,microwave distance increase,0.91456


In [6]:
new_df = pd.merge(df, df_corr, left_on='id', right_on='index', how='left')
new_df.drop(columns=['index'], inplace=True)
print(len(new_df))
new_df.head(3)

126


,id,segment,cmd,closest_pattern,predicted_feature,confidence_score,predicted,truth
0,1,1,move up,Move up,Z-cartesian increase,0.97794,"[microwave distance increase, plate distance d...","[microwave distance decrease, plate distance d..."
1,1,2,closer to the plate,Stay close to plate,plate distance decrease,0.91750,"[microwave distance increase, plate distance d...","[microwave distance decrease, plate distance d..."
2,1,3,microwave,Avoid microwave,microwave distance increase,0.91456,"[microwave distance increase, plate distance d...","[microwave distance decrease, plate distance d..."


In [7]:
new_df['id'] = new_df['id'].astype(int)  
new_df['predicted'] = new_df['predicted'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)  
new_df['truth'] = new_df['truth'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)  
  
df = new_df.groupby('id', as_index=False).agg({  
    'segment': list,  
    'cmd': list,  
    'closest_pattern': list,  
    'predicted_feature': list,  
    'confidence_score': list,  
    'predicted': 'first',  
    'truth': 'first'  
})  

df.sort_values(by='id', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

df.head(6)

,id,segment,cmd,closest_pattern,predicted_feature,confidence_score,predicted,truth
0,1,"[1, 2, 3]","[move up, closer to the plate, microwave]","[Move up, Stay close to plate, Avoid microwave]","[Z-cartesian increase, plate distance decrease...","[0.97794, 0.9175, 0.91456]","microwave distance increase, plate distance de...","microwave distance decrease, plate distance de..."
1,2,"[1, 2]","[Move near the cup, Microwave]","[Move closer to cup, Up]","[cup distance decrease, Z-cartesian increase]","[0.97965, 0.80242]","cup distance decrease, z-cartesian increase","cup distance decrease, microwave distance decr..."
2,3,"[1, 2, 3]","[move up, further away from the bowl, further ...","[Move up, Move further away from bowl, Move fu...","[Z-cartesian increase, bowl distance increase,...","[0.97794, 0.94585, 0.94382]",None,None
3,4,"[1, 2, 3]","[stay closer to the bottle, stay closer to the...","[Stay close to bottle, Stay close to apple, Mo...","[bottle distance decrease, apple distance decr...","[0.96689, 0.95993, 0.92768]",None,None
4,5,"[1, 2]","[Get close to the bottle, Get close to the not...","[Stay close to bottle, Stay close to notebook]","[bottle distance decrease, notebook distance d...","[0.95601, 0.95944]",None,None
5,6,"[1, 2]","[Get close to the orange, Get close to the not...","[Stay close to orange, Stay close to notebook]","[orange distance decrease, notebook distance d...","[0.96248, 0.95945]",None,None


In [8]:
df.to_excel(f"./{embedding_type}.xlsx", index=False)

In [11]:
import pandas as pd

df = pd.read_excel(f"./{embedding_type}.xlsx")
df.set_index('id', inplace=True)
df['match'] = df['truth'].isna()
df.head(6)

,segment,cmd,closest_pattern,predicted_feature,confidence_score,predicted,truth,match
id,,,,,,,,
1,"[1, 2, 3]","['move up', 'closer to the plate', 'microwave']","['Move up', 'Stay close to plate', 'Avoid micr...","['Z-cartesian increase', 'plate distance decre...","[0.97794, 0.9175, 0.91456]","microwave distance increase, plate distance de...","microwave distance decrease, plate distance de...",False
2,"[1, 2]","['Move near the cup', 'Microwave']","['Move closer to cup', 'Up']","['cup distance decrease', 'Z-cartesian increase']","[0.97965, 0.80242]","cup distance decrease, z-cartesian increase","cup distance decrease, microwave distance decr...",False
3,"[1, 2, 3]","['move up', 'further away from the bowl', 'fur...","['Move up', 'Move further away from bowl', 'Mo...","['Z-cartesian increase', 'bowl distance increa...","[0.97794, 0.94585, 0.94382]",NaN,NaN,True
4,"[1, 2, 3]","['stay closer to the bottle', 'stay closer to ...","['Stay close to bottle', 'Stay close to apple'...","['bottle distance decrease', 'apple distance d...","[0.96689, 0.95993, 0.92768]",NaN,NaN,True
5,"[1, 2]","['Get close to the bottle', 'Get close to the ...","['Stay close to bottle', 'Stay close to notebo...","['bottle distance decrease', 'notebook distanc...","[0.95601, 0.95944]",NaN,NaN,True
6,"[1, 2]","['Get close to the orange', 'Get close to the ...","['Stay close to orange', 'Stay close to notebo...","['orange distance decrease', 'notebook distanc...","[0.96248, 0.95945]",NaN,NaN,True


In [13]:
df['match'].value_counts()

match
True     41
False     9
Name: count, dtype: int64

In [14]:
df_stat = pd.DataFrame(df['match'].value_counts())
df_stat

,count
match,
True,41
False,9


In [12]:
markdown_table = df.to_markdown(buf=None, mode='pipe', index=True)
res_table = df_stat.to_markdown(buf=None, mode='pipe', index=True)
with open(f'./{embedding_type}.md', 'w') as file:
    file.write("## Result - ADA (with prompts)\n\n")
    file.write(markdown_table)
    file.write("\n\n")
    file.write("## Stats\n\n")
    file.write(res_table)